In [1]:
!nvidia-smi
!free -g
!pwd
!ls
!lscpu

Thu Aug  5 09:44:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    22W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# https://drive.google.com/file/d/1v4mz7sezY1ldqH4BB8Q1OFDQzb_Y_Puo/view?usp=sharing
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1v4mz7sezY1ldqH4BB8Q1OFDQzb_Y_Puo' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1v4mz7sezY1ldqH4BB8Q1OFDQzb_Y_Puo" -O baseline2107.zip && rm -rf /tmp/cookies.txt

--2021-08-05 09:44:31--  https://docs.google.com/uc?export=download&confirm=RzHz&id=1v4mz7sezY1ldqH4BB8Q1OFDQzb_Y_Puo
Resolving docs.google.com (docs.google.com)... 142.251.2.101, 142.251.2.113, 142.251.2.138, ...
Connecting to docs.google.com (docs.google.com)|142.251.2.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-a0-docs.googleusercontent.com/docs/securesc/q6vfborbn7gd4s0cmo31euh03r58e7d0/klkij5ao0mevldq08vajcu5qvqgje0uu/1628156625000/04675350653884988735/14692938404411821174Z/1v4mz7sezY1ldqH4BB8Q1OFDQzb_Y_Puo?e=download [following]
--2021-08-05 09:44:32--  https://doc-08-a0-docs.googleusercontent.com/docs/securesc/q6vfborbn7gd4s0cmo31euh03r58e7d0/klkij5ao0mevldq08vajcu5qvqgje0uu/1628156625000/04675350653884988735/14692938404411821174Z/1v4mz7sezY1ldqH4BB8Q1OFDQzb_Y_Puo?e=download
Resolving doc-08-a0-docs.googleusercontent.com (doc-08-a0-docs.googleusercontent.com)... 142.250.141.132, 2607:f8b0:4023:c0b::84
Connecting to

In [3]:
!rm -rf PavementDataResize2107
!unzip baseline2107.zip > /dev/null 2>&1

In [4]:
import os
os.listdir('./PavementDataResize2107')

['.tmp.drivedownload',
 'longitudinal_crack',
 'mend',
 'normal',
 'transverse_crack']

In [5]:
!rm -rf ./PavementDataResize2107/.tmp.drivedownload
os.listdir('./PavementDataResize2107')

['longitudinal_crack', 'mend', 'normal', 'transverse_crack']

In [6]:
import sys
import pickle
import numpy as np
import torch
from tqdm.notebook import tqdm
from PIL import Image
from sklearn.metrics import classification_report
import pandas as pd
import os

def path_exist(path):
    if os.path.exists(path) is False:
        os.mkdir(path)

def write_pickle(list_info: list, file_name: str):
    with open(file_name, 'wb') as f:
        pickle.dump(list_info, f)
    f.close()


def read_pickle(file_name: str) -> list:
    with open(file_name, 'rb') as f:
        info_list = pickle.load(f)
    f.close()
    return info_list

def rgb_loader(path: str):
    return Image.open(path).convert('RGB')

def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    loss_function = torch.nn.CrossEntropyLoss()
    # 累计损失
    accu_loss = torch.zeros(1).to(device)
    # 累计预测正确的样本数目
    accu_num = torch.zeros(1).to(device)
    optimizer.zero_grad()

    sample_num = 0
    data_loader = tqdm(data_loader)
    for step, data in enumerate(data_loader):
        images, labels = data
        sample_num += images.shape[0]

        pred = model(images.to(device))
        pred_classes = torch.max(pred, dim=1)[1]
        accu_num += torch.eq(pred_classes, labels.to(device)).sum()

        loss = loss_function(pred, labels.to(device))
        loss.backward()
        accu_loss += loss.detach()

        data_loader.desc = "[train epoch {}] loss: {:.6f}, acc: {:.6f}.".format(
            epoch,
            accu_loss.item() / (step + 1),
            accu_num.item() / sample_num)

        if not torch.isfinite(loss):
            print("WARNING: no-finite loss, ending training ", loss)
            sys.exit(1)

        optimizer.step()
        optimizer.zero_grad()
    return accu_loss.item() / (step + 1), accu_num.item() / sample_num


@torch.no_grad()
def evaluate(model, data_loader, device, epoch):
    loss_function = torch.nn.CrossEntropyLoss()
    model.eval()

    # 累计损失
    accu_loss = torch.zeros(1).to(device)
    # 累计预测正确的样本数目
    accu_num = torch.zeros(1).to(device)

    sample_num = 0

    data_loader = tqdm(data_loader)
    for step, data in enumerate(data_loader):
        images, labels = data
        sample_num += images.shape[0]

        pred = model(images.to(device))
        pred_classes = torch.max(pred, dim=1)[1]
        accu_num += torch.eq(pred_classes, labels.to(device)).sum()

        loss = loss_function(pred, labels.to(device))
        accu_loss += loss

        data_loader.desc = "[validation epoch {}] loss: {:.6f}, acc: {:.6f}.".format(
            epoch,
            accu_loss.item() / (step + 1),
            accu_num.item() / sample_num)

    return accu_loss.item() / (step + 1), accu_num.item() / sample_num

@torch.no_grad()
def test_model(model, data_loader, device, epoch, class_labels:list):
    loss_function = torch.nn.CrossEntropyLoss()
    model.eval()

    # 累计损失
    accu_loss = torch.zeros(1).to(device)
    # 累计预测正确的样本数目
    accu_num = torch.zeros(1).to(device)

    sample_num = 0
    test_result = {'labels': np.array([], dtype='u1'), 'preds': np.array([], dtype='u1')}
    data_loader = tqdm(data_loader)
    for step, data in enumerate(data_loader):
        images, labels = data
        test_result['labels'] = np.concatenate((test_result['labels'], np.array(labels, dtype='u1')))
        sample_num += images.shape[0]

        pred = model(images.to(device))
        pred_classes = torch.max(pred, dim=1)[1]
        accu_num += torch.eq(pred_classes, labels.to(device)).sum()
        test_result['preds'] = np.concatenate((test_result['preds'], np.array(pred_classes.cpu(), dtype='u1')))

        loss = loss_function(pred, labels.to(device))
        accu_loss += loss

        data_loader.desc = "[test epoch {}] loss: {:.6f}, acc: {:.6f}.".format(
            epoch,
            accu_loss.item() / (step + 1),
            accu_num.item() / sample_num)
        result = classification_report(test_result['labels'], test_result['preds'], target_names=class_labels, zero_division=0, output_dict=True, digits=6)
        # print(classification_report(test_result['labels'], test_result['preds'], target_names=class_labels, zero_division=0, digits=6))

    return accu_loss.item() / (step + 1), accu_num.item() / sample_num, pd.DataFrame(result).transpose()

In [7]:
!nvidia-smi
!pwd

Thu Aug  5 09:44:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
%tb

import warnings
import os
import math
import pandas as pd
import json
import argparse

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.data import random_split

warnings.filterwarnings('ignore')

import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.tensorboard import SummaryWriter
import torchvision.models as models


# from utils import rgb_loader, train_one_epoch, evaluate, test_model

os.environ["CUDA_VISIBLE_DEVICES"] = '0'


torch.manual_seed(90)

def dataset_split(dataset, split_rate:float = 0.2):
    dataset_len = len(dataset)
    lengths = [dataset_len - int(dataset_len * split_rate), int(dataset_len * split_rate)]
    # 设置seed，确保随机结果的可复现性
    left_dataset, split_dataset = random_split(dataset=dataset, lengths=lengths)
    return left_dataset, split_dataset

def dataset_split_by_path(data_path, transform, loader=rgb_loader, split_rate:float = 0.2):
    dataset = ImageFolder(root=data_path, transform=transform, loader=loader)
    return dataset_split(dataset, split_rate)


def generate_class_json(root_path:str):
    # 遍历文件夹，一个文件夹对应一个类别
    number_class = [
        cla for cla in os.listdir(root_path)
        if os.path.isdir(os.path.join(root_path, cla))
    ]
    # 排序，保证顺序一致性
    number_class.sort()
    # 生成类别名称以及对应的数字索引
    class_indices = dict((k, v) for v, k in enumerate(number_class))
    json_str = json.dumps(dict(
        (val, key) for key, val in class_indices.items()),indent=4)
    with open("class_indices.json", "w") as json_file:
        json_file.write(json_str)
    json_file.close()


def model_modify(model, num_classes:int):
    model_name = model._get_name().lower()
    if model_name == 'resnet':
        features = model.fc.in_features
        model.fc = nn.Linear(features, num_classes, bias=True)
    elif model_name == 'vgg':
        features = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(features, num_classes, bias=True)
    elif model_name == 'googlenet':
        features = model.fc.in_features
        model.fc = nn.Linear(features, num_classes, bias=True)
    else:
        raise Exception("Model `{}` modify code does not exits!, Must Change Code of Function `model_modify`!".format(model_name))
    return model


def main(args):

    device = torch.device(args.device if torch.cuda.is_available() else 'cpu')

    data_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.22442342, 0.22442342, 0.22442342], [0.079417765, 0.079417765, 0.079417765])
        ])
    
    if os.path.exists('../weight') is False:
        os.makedirs("../weight")
    
    path_exist(args.log_path)
    tb_writer = SummaryWriter(args.log_path)
    num_workers = args.nw if args.nw > 0 else int(os.cpu_count()/2)
    batch_size = args.batch_size
    labels = os.listdir(args.data_path)
    test_dataframe = pd.DataFrame()

    # 准备数据集
    train_dataset, test_dataset = dataset_split_by_path(args.data_path, transform=data_transform, loader=rgb_loader, split_rate=0.2)
    test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, pin_memory=True, num_workers=num_workers)

    # 准备好模型
    model = models.googlenet(pretrained=True)
    model = model_modify(model, args.num_classes)
    model = model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=args.lr, weight_decay=0.001)

    # Scheduler https://arxiv.org/pdf/1812.01187.pdf 
    lf = lambda x: ((1 + math.cos(x * math.pi / args.epoch)) / 2) * (1 - args.lrf) + args.lrf  # cosine
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lf)

    for epoch in range(args.epoch):
        epoch_train_dataset, epoch_val_dataset = dataset_split(train_dataset, 0.2)
        train_dataloader = DataLoader(dataset = epoch_train_dataset, batch_size=batch_size, pin_memory=True, num_workers=num_workers)
        val_dataloader = DataLoader(dataset=epoch_val_dataset, batch_size=batch_size, pin_memory=True, num_workers=num_workers)
        
        # train
        train_loss, train_acc = train_one_epoch(
            model=model,
            optimizer=optimizer,
            data_loader=train_dataloader,
            device=device,
            epoch=epoch
        )

        scheduler.step()

        # validate
        val_loss, val_acc = evaluate(
            model=model,
            data_loader = val_dataloader,
            device = device,
            epoch = epoch
        )

        # test
        test_loss, test_acc, epoch_test_dataframe = test_model(
            model=model,
            data_loader = test_dataloader,
            device = device,
            epoch = epoch,
            class_labels = labels
        )

        test_dataframe = test_dataframe.append(pd.DataFrame(epoch_test_dataframe))
        test_dataframe.to_csv('./' + model._get_name().lower() + "_test_result.csv", index=True)

        tags = ['train_loss', 'train_acc', 'val_loss', 'val_acc', 'test_loss', 'test_acc', 'learning_rate']
        tb_writer.add_scalar(tags[0], train_loss, epoch)
        tb_writer.add_scalar(tags[1], train_acc, epoch)
        tb_writer.add_scalar(tags[2], val_loss, epoch)
        tb_writer.add_scalar(tags[3], val_acc, epoch)
        tb_writer.add_scalar(tags[4], test_loss, epoch)
        tb_writer.add_scalar(tags[5], test_acc, epoch)
        tb_writer.add_scalar(tags[6], optimizer.param_groups[0]['lr'], epoch)

        # 保存每轮训练完之后的模型权重
        # torch.save(model, os.path.join(args.save_path, model._get_name().lower() + '_epoch_' + str(epoch) + '_.pth'))

# if __name__ == '__main__':
parser = argparse.ArgumentParser()

parser.add_argument('--num_classes', type=int, default=4)
parser.add_argument('--epoch', type=int, default=30)
parser.add_argument('--nw', type=int, default=6)
parser.add_argument('--batch_size', type=int, default=128)
parser.add_argument('--lr', type=float, default=0.01)
parser.add_argument('--lrf', type=float, default=0.01)

# 数据集所在目录
parser.add_argument('--data_path', type=str, default='./PavementDataResize2107')

# 预训练权重
# parser.add_argument('--weight', type=str, default='./googlenet/googlenet.pth', help='initial weights path')
# parser.add_argument('--save_path', type=str, default='./weight/')
parser.add_argument('--log_path', type=str, default='./log/')

parser.add_argument('--device', default='cuda', help='device id(i.e. 0 or 0, 1 or cpu)')

opt = parser.parse_args(args=[])
main(opt)

No traceback available to show.


In [11]:
!zip googlenetlog.zip ./log/ -q -r